In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from datetime import datetime
from rasterstats import zonal_stats
from dbfread import DBF
import re



In [ ]:

# -----------------------
# Params
# -----------------------
project_db   = r"C:\Users\bsf31\Documents\data\NL060\NL060.gpkg"
faces_layer  = "California_Fir_FeatureToPoly"   # non-overlapping faces (Feature to Polygon)
perims_layer = "California_Fire_Perimet_Clip"   # original clipped perimeters
n_years      = 20                                # "recent" window

# Your known columns on the perimeters layer
year_col = "YEAR_"
name_col = "FIRE_NAME"

In [ ]:
# -----------------------
# Load
# -----------------------
faces  = gpd.read_file(project_db, layer=faces_layer)
perims = gpd.read_file(project_db, layer=perims_layer)

# CRS match
if faces.crs != perims.crs:
    perims = perims.to_crs(faces.crs)

# Coerce year to numeric (safe even if already numeric)
perims[year_col] = pd.to_numeric(perims[year_col], errors="coerce")

# Quick geom repair on perimeters (faces from FeatureToPoly are usually fine)
perims["geometry"] = perims.geometry.buffer(0)

# Ensure faces have a stable id
if "piece_id" not in faces.columns:
    faces = faces.copy()
    faces["piece_id"] = np.arange(len(faces), dtype=int)

In [ ]:

# -----------------------
# Area-based intersection (no edge-only matches), NO sliver filtering
# -----------------------
faces_slim  = faces[["piece_id", "geometry"]].copy()
perims_slim = perims[[year_col, name_col, "geometry"]].copy()

inter = gpd.overlay(faces_slim, perims_slim, how="intersection", keep_geom_type=True)

if inter.empty:
    out = faces.copy()
    out["LATEST_YEAR"] = pd.Series(dtype="Int64")
    out["LATEST_FIRE"] = pd.Series(dtype="object")
    out["BURN_COUNT"]  = 0
    out["HAS_OVERLAP"] = 0
    cutoff = datetime.now().year - n_years
    out[f"RECENT_{n_years}Y"] = 0
else:
    # Count how many perimeters truly overlap each face
    counts = inter.groupby("piece_id")[year_col].size().rename("BURN_COUNT")

    # Pick the perimeter row with MAX year per face
    top = (
        inter.sort_values(["piece_id", year_col], ascending=[True, False])
            .drop_duplicates(subset="piece_id")
            .loc[:, ["piece_id", year_col, name_col]]
            .rename(columns={year_col: "LATEST_YEAR", name_col: "LATEST_FIRE"})
    )

    # Merge onto faces
    out = faces.merge(top, on="piece_id", how="left").merge(counts, on="piece_id", how="left")

    # Flags (store as 0/1 for GPKG friendliness)
    out["BURN_COUNT"]  = out["BURN_COUNT"].fillna(0).astype("int32")
    out["HAS_OVERLAP"] = (out["BURN_COUNT"] > 1).astype("uint8")

    cutoff = datetime.now().year - n_years   # with 2025 and 20 yrs => 2005
    out[f"RECENT_{n_years}Y"] = out["LATEST_YEAR"].ge(cutoff).fillna(False).astype("uint8")

In [ ]:

# -----------------------
# Write a clean schema
# -----------------------
keep_cols   = ["piece_id", "geometry", "BURN_COUNT", "HAS_OVERLAP", "LATEST_YEAR", "LATEST_FIRE", f"RECENT_{n_years}Y"]
out_to_write = out[keep_cols].copy()

out_layer = f"{faces_layer}_tagged"
out_to_write.to_file(project_db, layer=out_layer, driver="GPKG")
print(f"✅ Wrote tagged faces to layer: {out_layer}")


# Zonal Stats Firesheds/ Scott & Burgan 40 Models

In [ ]:
# -----------------------
# Inputs and Load
# -----------------------
project_db   = r"C:\Users\bsf31\Documents\data\NL060\NL060.gpkg"
sb_firesheds = r"C:\Users\bsf31\Documents\data\NL060\sb_firesheds_v02a_20250821.gpkg"
dbf_path    = r"C:\Users\bsf31\Documents\data\NL060\landfire_meszxc7dfpgmqh\LF2024_FBFM40_250_CONUS\LF24_F40_250.dbf"
raster_path  = r"C:\Users\bsf31\Documents\data\NL060\landfire_meszxc7dfpgmqh\LF2024_FBFM40_250_CONUS\LC24_F40_250.tif"

latest_fire_name  = "latest_fire_name_alisal"   
sb_fireshed = 'alisal_buffer'

alisal_fire = gpd.read_file(project_db, layer=latest_fire_name)
firesheds = gpd.read_file(project_db, layer=sb_fireshed)




In [ ]:
meta = {
    # --- GRASS ---
    "GR1": ("Grass", "Arid–semiarid (EMC 15%)", "Short, patchy, possibly grazed; spread moderate; flame low."),
    "GR2": ("Grass", "Arid–semiarid (EMC 15%)", "Moderately coarse, ~1 ft; spread high; flame moderate."),
    "GR4": ("Grass", "Arid–semiarid (EMC 15%)", "Moderately coarse, ~2 ft; spread very high; flame high."),
    "GR7": ("Grass", "Arid–semiarid (EMC 15%)", "Moderately coarse, ~3 ft; spread very high; flame very high."),
    "GR3": ("Grass", "Subhumid–humid (EMC 30–40%)", "Very coarse, ~2 ft; spread high; flame moderate."),
    "GR5": ("Grass", "Subhumid–humid (EMC 30–40%)", "Dense, coarse, 1–2 ft; spread very high; flame high."),
    "GR6": ("Grass", "Subhumid–humid (EMC 30–40%)", "Dryland grass 1–2 ft; spread very high; flame very high."),
    "GR8": ("Grass", "Subhumid–humid (EMC 30–40%)", "Heavy, coarse, 3–5 ft; spread very high; flame very high."),
    "GR9": ("Grass", "Subhumid–humid (EMC 30–40%)", "Very heavy, coarse, 5–8 ft; spread extreme; flame extreme."),
    # --- GRASS-SHRUB ---
    "GS1": ("Grass-Shrub", "Arid–semiarid (EMC 15%)", "Shrubs ~1 ft, low grass; spread moderate; flame low."),
    "GS2": ("Grass-Shrub", "Arid–semiarid (EMC 15%)", "Shrubs 1–3 ft, moderate grass; spread high; flame moderate."),
    "GS3": ("Grass-Shrub", "Subhumid–humid (EMC 30–40%)", "Moderate grass/shrub <2 ft; spread high; flame moderate."),
    "GS4": ("Grass-Shrub", "Subhumid–humid (EMC 30–40%)", "Heavy grass/shrub >2 ft; spread high; flame very high."),
    # --- SHRUB ---
    "SH1": ("Shrub", "Arid–semiarid (EMC 15%)", "Low shrub load ~1 ft; spread very low; flame very low."),
    "SH2": ("Shrub", "Arid–semiarid (EMC 15%)", "Moderate load ~1 ft; no grass; spread low; flame low."),
    "SH5": ("Shrub", "Arid–semiarid (EMC 15%)", "Heavy shrubs 4–6 ft; spread very high; flame very high."),
    "SH7": ("Shrub", "Arid–semiarid (EMC 15%)", "Very heavy shrubs 4–6 ft; spread high; flame very high."),
    "SH3": ("Shrub", "Subhumid–humid (EMC 30–40%)", "Moderate shrubs (maybe pine/herb); 2–3 ft; spread low; flame low."),
    "SH4": ("Shrub", "Subhumid–humid (EMC 30–40%)", "Low–moderate shrubs/litter (~3 ft); spread high; flame moderate."),
    "SH6": ("Shrub", "Subhumid–humid (EMC 30–40%)", "Dense shrubs, little/no herb; ~2 ft; spread high; flame high."),
    "SH8": ("Shrub", "Subhumid–humid (EMC 30–40%)", "Dense shrubs, ~3 ft; spread high; flame high."),
    "SH9": ("Shrub", "Subhumid–humid (EMC 30–40%)", "Dense, fine-branched, 4–6 ft; spread high; flame very high."),
    # --- TIMBER-UNDERSTORY ---
    "TU1": ("Timber-Understory", "Semiarid–subhumid (EMC 20%)", "Low grass/shrub + litter; spread low; flame low."),
    "TU2": ("Timber-Understory", "Humid (EMC 30%)", "Moderate litter with shrubs; spread moderate; flame low."),
    "TU3": ("Timber-Understory", "Humid (EMC 30%)", "Moderate litter + grass/shrubs; spread high; flame moderate."),
    "TU4": ("Timber-Understory", "Semiarid–subhumid (EMC 20%)", "Short conifers w/ grass/moss; spread moderate; flame moderate."),
    "TU5": ("Timber-Understory", "Semiarid–subhumid (EMC 20%)", "High conifer litter + shrubs; spread moderate; flame moderate."),
    # --- TIMBER LITTER ---
    "TL1": ("Timber Litter", "Recently burned", "Light–moderate load, 1–2 in deep; spread very low; flame very low."),
    "TL2": ("Timber Litter", "Broadleaf litter", "Low load, compact; spread very low; flame very low."),
    "TL3": ("Timber Litter", "Other conifer litter", "Moderate conifer litter; spread very low; flame low."),
    "TL4": ("Timber Litter", "Mixed fine & coarse", "Moderate load incl. small logs; spread low; flame low."),
    "TL5": ("Timber Litter", "Conifer litter", "High load; light slash/mortality; spread low; flame low."),
    "TL6": ("Timber Litter", "Broadleaf litter", "Moderate load, less compact; spread moderate; flame low."),
    "TL7": ("Timber Litter", "Mixed fine & coarse", "Heavy load incl. larger logs; spread low; flame low."),
    "TL8": ("Timber Litter", "Long-needle pine", "Moderate load/compact; some herb; spread moderate; flame low."),
    "TL9": ("Timber Litter", "Broadleaf / needle drape", "Very high load; spread moderate; flame moderate."),
    # --- SLASH/BLOWDOWN ---
    "SB1": ("Slash-Blowdown", "Activity fuel", "10–20 t/ac; fuels 1–3 in; <1 ft depth; spread moderate; flame low."),
    "SB2": ("Slash-Blowdown", "Activity fuel", "7–12 t/ac; even 0–3 in; ~1 ft; spread moderate; flame moderate."),
    "SB3": ("Slash-Blowdown", "Activity fuel", "7–12 t/ac; weighted to <0.25 in; >1 ft; spread high; flame high."),
    # blowdown variants share codes with SB2/SB3/SB4 as behavior descriptors
    "SB4": ("Slash-Blowdown", "Blowdown (total)", "Total blowdown; not compacted; foliage attached; spread very high; flame very high."),
    # --- NONBURNABLE ---
    "NB1": ("Nonburnable", "—", "Urban/suburban; insufficient wildland fuel."),
    "NB2": ("Nonburnable", "—", "Snow/ice."),
    "NB3": ("Nonburnable", "—", "Agricultural field maintained nonburnable."),
    "NB8": ("Nonburnable", "—", "Open water."),
    "NB9": ("Nonburnable", "—", "Bare ground."),
}

In [ ]:

fbfm40_meta = (
    pd.DataFrame.from_dict(meta, orient="index", columns=["general_type", "climate", "summary"])
      .reset_index()
      .rename(columns={"index": "MODEL"})
)

In [ ]:
# Only those with the recent fire flag set
#recent_faces = tagged[tagged["RECENT_20Y"] == 1].copy()

# -----------------------
# Dissolve by LATEST_FIRE
# -----------------------
# One polygon per fire name, but only for flagged ones
#groups = recent_faces.dissolve(by="LATEST_FIRE", as_index=False)


In [ ]:
# 1) Read DBF to DataFrame
tbl = DBF(dbf_path, load=True, char_decode_errors='ignore')
lut_df = pd.DataFrame(iter(tbl))
value_col = 'VALUE'
model_col = "FBFM40"
# Keep just code + label; standardize names
lut_df = lut_df[[value_col, model_col]].copy()
lut_df.columns = ["VALUE", "FBFM40"]
lut_df = lut_df.rename(columns={"FBFM40": "MODEL"})


In [ ]:
lut_df

# Zonal Stats

In [ ]:
# -----------------------
# Run zonal statistics
# -----------------------
zs = zonal_stats(
    firesheds,
    raster_path,
    categorical=True,   # return pixel counts per EVT code
    nodata=-9999)

# 3) Build long table from your zs (fid, VALUE, pixels)
rows = []
for fid, counts in enumerate(zs):
    for val, cnt in counts.items():
        rows.append({"fid": fid, "VALUE": int(val), "pixels": int(cnt)})
df_counts = pd.DataFrame(rows)
# 4) Join codes → names
df_counts = df_counts.merge(lut_df, on="VALUE", how="left")
df_counts = df_counts.merge(fbfm40_meta, on="MODEL", how="left")
df_counts

In [ ]:
""" 
# Attach results back to the GeoDataFrame
firesheds["fbfm40_unique"] = [list(s.keys()) for s in zs]
firesheds[["fbfm40_unique"]]

# 5) Attach unique lists back to firesheds
uniq_codes = (
    df_counts.groupby("fid")["VALUE"]
    .apply(lambda s: sorted(set(s.tolist())))
    .rename("fbfm40_unique_codes")
)
uniq_names = (
    df_counts.groupby("fid")["FBFM40_LABEL"]
    .apply(lambda s: sorted(set([x for x in s.tolist() if pd.notnull(x)])))
    .rename("fbfm40_unique_names")
)

firesheds = firesheds.join(uniq_codes, how="left")
firesheds = firesheds.join(uniq_names, how="left") """

In [ ]:
# -----------------------
# Save
# -----------------------
""" out_layer = f"{faces_layer}_zonal_byfire_recent"
out.to_file(project_db, layer=out_layer, driver="GPKG")

print("✅ Zonal stats by fire (RECENT_20Y == 1) complete")
print(out[["LATEST_FIRE", "count", "majority_evt"]].head()) """

# Scott & Burgan 40 metadata Fields: general_type, climate, summary (short description)

In [ ]:
# -----------------------
# Run zonal statistics
# -----------------------
zs_lf = zonal_stats(
    alisal_fire,
    raster_path,
    categorical=True,   # return pixel counts per EVT code
    nodata=-9999)

# 3) Build long table from  zs_latest_fire (fid, VALUE, pixels)
rows = []
for fid, counts in enumerate(zs_lf):
    for val, cnt in counts.items():
        rows.append({"fid": fid, "VALUE": int(val), "pixels": int(cnt)})
df_counts_lf = pd.DataFrame(rows)
# 4) Join codes → names
df_counts_lf = df_counts_lf.merge(lut_df, on="VALUE", how="left")
df_counts_lf = df_counts_lf.merge(fbfm40_meta, on="MODEL", how="left")
df_counts_lf

In [ ]:
FAMILY_ORDERS = {
    'GR': ['GR1','GR2','GR3','GR4','GR5','GR6','GR7','GR8','GR9'],
    'GS': ['GS1','GS2','GS3','GS4'],
    'SH': ['SH1','SH2','SH3','SH4','SH5','SH6','SH7','SH8','SH9'],
    'TU': ['TU1','TU2','TU3','TU4','TU5'],
    'TL': ['TL1','TL2','TL3','TL4','TL5','TL6','TL7','TL8','TL9'],
    'SB': ['SB1','SB2','SB3','SB4'],
    'NB': ['NB1','NB2','NB3','NB8','NB9'],
}

In [ ]:
def clean_model(s):
    """Normalize to code like 'GR4' from text. Returns None if not parseable."""
    if pd.isna(s):
        return None
    m = re.search(r'\b([A-Z]{2}\d{1,2})\b', str(s).upper())
    return m.group(1) if m else None

def family(code):
    return code[:2] if isinstance(code, str) and len(code) >= 2 else None

def model_to_preferred_value(df_counts):
    ref = df_counts.copy()
    ref["MODEL_ref"] = ref["MODEL"].map(clean_model)
    mv = (
        ref.groupby(["MODEL_ref","VALUE"], dropna=False)["pixels"]
           .sum()
           .reset_index()
    )
    idx = mv.groupby("MODEL_ref")["pixels"].idxmax()
    best = mv.loc[idx, ["MODEL_ref","VALUE"]].dropna()
    return dict(zip(best["MODEL_ref"], best["VALUE"]))

def build_allowed_by_family(df_counts):
    """
    From big-area df_counts, find which models are present (>0 pixels) per family.
    Return ordered lists consistent with FAMILY_ORDERS.
    """
    ref = df_counts.copy()
    ref["MODEL_ref"] = ref["MODEL"].map(clean_model)
    model_pixels = ref.groupby("MODEL_ref", dropna=False)["pixels"].sum()

    present = {fam: set() for fam in FAMILY_ORDERS.keys()}
    for m, pix in model_pixels.items():
        if not isinstance(m, str) or pix <= 0:
            continue
        fam = family(m)
        if fam in FAMILY_ORDERS:
            present[fam].add(m)

    allowed_by_family = {
        fam: [x for x in FAMILY_ORDERS[fam] if x in present[fam]]
        for fam in FAMILY_ORDERS.keys()
    }
    return allowed_by_family


In [ ]:
def build_ring_type_modal(df_counts_ring, lut_df, fbfm40_meta):
    """
    Returns a DataFrame with the dominant (modal) MODEL per general_type in the ring:
      columns: general_type | MODEL_modal
    """


    ring = df_counts_ring.copy()
    ring = ring[['VALUE','pixels']].dropna()
    ring['VALUE'] = pd.to_numeric(ring['VALUE'], errors='coerce').astype('Int64')

    # VALUE -> MODEL, then attach general_type
    ring = ring.merge(lut_df, on='VALUE', how='left').merge(fbfm40_meta, on='MODEL', how='left')

    # Sum pixels per (general_type, MODEL), then take the modal MODEL per type
    ring_tot = (ring.groupby(['general_type','MODEL'], dropna=False)['pixels']
                    .sum().reset_index())
    ring_tot = ring_tot.sort_values(['general_type','pixels'], ascending=[True, False])

    ring_type_modal = (ring_tot.groupby('general_type', as_index=False)
                              .first()[['general_type','MODEL']]
                              .rename(columns={'MODEL':'MODEL_modal'}))
    return ring_type_modal

In [ ]:
# --- Apply ring modal mapping (type-constrained) to burned table ---
def upgrade_to_ring_type_modal(df_counts_lf, df_counts_reference, ring_type_modal):
    """
    Map each burned record to the ring's modal MODEL of the same general_type.
    VALUE_after comes from the reference's canonical VALUE for that MODEL.
    """
    out = df_counts_lf.copy()

    # Ensure MODEL_before is clean code
    out['MODEL_before'] = out['MODEL'].map(clean_model)
    out = out.rename(columns={'VALUE':'VALUE_before'})

    # Build canonical VALUE per MODEL from reference (unburned)
    model_value_map = model_to_preferred_value(df_counts_reference)

    # Join by general_type to get the ring's modal MODEL for that type
    out = out.merge(ring_type_modal, on='general_type', how='left')

    # If no modal found for that type (rare), keep original model
    out['MODEL_after'] = out['MODEL_modal'].fillna(out['MODEL_before'])
    out.drop(columns=['MODEL_modal'], inplace=True)

    # VALUE_after via canonical mapping; fallback to VALUE_before if missing
    out['VALUE_after'] = out['MODEL_after'].map(model_value_map).fillna(out['VALUE_before'])

    # Optional neat column order
    front = ['fid','MODEL_before','VALUE_before','MODEL_after','VALUE_after','general_type','climate','summary','pixels']
    out = out[[c for c in front if c in out.columns] + [c for c in out.columns if c not in front]]

    # Quick summaries if you want them:
    before = out.groupby('MODEL_before', dropna=False)['pixels'].sum().sort_values(ascending=False)
    after  = out.groupby('MODEL_after',  dropna=False)['pixels'].sum().sort_values(ascending=False)

    return out, before.rename('pixels_before'), after.rename('pixels_after')

In [ ]:
# 1) Build ring modal per general_type from your ring counts
# df_counts_ring must have: VALUE, pixels  (counts from the ring buffer)
ring_type_modal = build_ring_type_modal(df_counts_lf, lut_df, fbfm40_meta)

# 2) Apply mapping to burned table, with VALUE_after from unburned reference
upgraded_df_lf_ring, before, after = upgrade_to_ring_type_modal(
    df_counts_lf,       # burned counts (inside)
    df_counts,          # unburned reference (used only for canonical VALUEs)
    ring_type_modal
)

In [ ]:
ring_type_modal

In [ ]:
upgraded_df_lf_ring

In [ ]:
# 1) Grand totals must match
tot_before = upgraded_df_lf["pixels"].sum()
tot_after  = upgraded_df_lf.groupby("MODEL_after")["pixels"].sum().sum()
print("Grand totals equal?", tot_before == tot_after, tot_before, tot_after)

# 2) Family totals must match (we never cross families)
bf_fam = (upgraded_df_lf.assign(fam=upgraded_df_lf["MODEL_before"].str[:2])
          .groupby("fam")["pixels"].sum().sort_values(ascending=False))
af_fam = (upgraded_df_lf.assign(fam=upgraded_df_lf["MODEL_after"].str[:2])
          .groupby("fam")["pixels"].sum().sort_values(ascending=False))
print("Family totals (before):\n", bf_fam)
print("Family totals (after):\n",  af_fam)

# 3) See exactly how pixels flowed from before -> after (confusion matrix)
flow = (pd.crosstab(
            upgraded_df_lf["MODEL_before"],
            upgraded_df_lf["MODEL_after"],
            values=upgraded_df_lf["pixels"],
            aggfunc="sum",
            dropna=False,
        ).fillna(0).astype(int))
print(flow)

In [ ]:
reclass_df = (
    upgraded_df_lf[["VALUE_before", "VALUE_after"]]
      .dropna(subset=["VALUE_after"])
      .drop_duplicates()
      .astype(int)
      .sort_values(["VALUE_before", "VALUE_after"])
      .reset_index(drop=True)
)


In [ ]:
reclass_df

In [ ]:
df_counts

In [ ]:
df_counts_lf

In [ ]:
upgraded_df_lf_simple = upgraded_df_lf.drop(columns=['fid', 'pixels', 'general_type', 'climate', 'summary','MODEL'])


In [ ]:
upgraded_df_lf_simple

In [ ]:
remapped_df = upgraded_df_lf_simple.drop(columns=['MODEL_before', 'VALUE_before'])
remapped_df = remapped_df.rename(columns={'MODEL_after': 'MODEL', 'VALUE_after': 'VALUE'})
remapped_df

In [ ]:
fbfm40_meta

In [ ]:
print(fbfm40_meta)

In [ ]:
remapped_df = remapped_df.merge(fbfm40_meta, on="MODEL", how="left")

In [ ]:
df_counts_lf_clean = remapped_df.rename(columns={'MODEL_after': 'MODEL', 'VALUE_after': 'VALUE'})

df_counts_lf_clean[['general_type','MODEL', 'VALUE',  'climate', 'summary']]

In [ ]:
remapped_df

In [ ]:
# def map_nearest_equal_or_higher(model, allowed_by_family, strict=False, upgrade_nb = False):
#     """
#     If strict=False (default): map to nearest equal-or-higher rung present in reference.
#     If strict=True:            map to nearest strictly-higher rung if available; else keep current.
#     NBx models are unchanged.
#     """
#     m = clean_model(model)
#     if m is None:
#         return model
#     fam = family(m)
#     if fam not in FAMILY_ORDERS:
#         return m
#     if fam == "NB" and not upgrade_nb:
#         return m

#     allowed = allowed_by_family.get(fam, [])
#     if not allowed:
#         return m

#     try:
#         idx_current = FAMILY_ORDERS[fam].index(m)
#     except ValueError:
#         # unknown rung -> fall back to family's max present
#         return allowed[-1]

#     for cand in allowed:
#         idx_cand = FAMILY_ORDERS[fam].index(cand)
#         if (idx_cand > idx_current) if strict else (idx_cand >= idx_current):
#             return cand

#     # No higher (strict) or equal/higher (non-strict) found
#     return m if strict else allowed[-1]

# def upgrade_latest_fire_nearest(df_counts, df_counts_lf, strict=False, upgrade_nb= False):
#     """
#     Perform nearest_equal_or_higher upgrades using df_counts (reference) and
#     apply to df_counts_lf (latest fire).
#     Returns: (upgraded_df_lf, before_pixels, after_pixels, mapping_df)
#     """
#     allowed_by_family = build_allowed_by_family(df_counts)
#     model_value_map = model_to_preferred_value(df_counts)

#     out = df_counts_lf.copy()
#      # BEFORE / AFTER columns
#     out["MODEL_before"] = out["MODEL"].map(clean_model)         # normalized but human-readable
#     out["MODEL_after"]  = out["MODEL_before"].apply(lambda m: map_nearest_equal_or_higher(m, allowed_by_family, strict=strict, upgrade_nb=upgrade_nb))
#     out = out.rename(columns={"VALUE": "VALUE_before"})           # keep original numeric code
#     out["VALUE_after"]  = out["MODEL_after"].map(model_value_map) #  numeric code in reference

#     # Summaries (pixels by model)
#     before = (
#         out.groupby("MODEL_before", dropna=False)["pixels"]
#            .sum().sort_values(ascending=False)
#            .rename("pixels_before")
#     )
#     after = (
#         out.groupby("MODEL_after", dropna=False)["pixels"]
#            .sum().sort_values(ascending=False)
#            .rename("pixels_after")
#     )

#     # Mapping table (distinct models seen in latest fire)
#     mapping = (
#         out[["MODEL_before","MODEL_after"]]
#           .drop_duplicates()
#           .sort_values(["MODEL_before","MODEL_after"])
#           .reset_index(drop=True)
#     )
#     mapping["VALUE_pref_ref_before"] = mapping["MODEL_before"].map(model_value_map)
#     mapping["VALUE_pref_ref_after"]  = mapping["MODEL_after"].map(model_value_map)

#     # Reorder key columns up front (nice to view)
#     front = ["fid","MODEL_before","VALUE_before","MODEL_after","VALUE_after","pixels"]
#     cols = [c for c in front if c in out.columns] + [c for c in out.columns if c not in front]
#     out = out[cols]

#     return out, before, after, mapping
# upgraded_df_lf, before, after, mapping = upgrade_latest_fire_nearest(
#     df_counts=df_counts,
#     df_counts_lf=df_counts_lf,
#     strict=True,
#     upgrade_nb= False
# )
# print(mapping)
# print(before.head(10))
# print(after.head(10))
# upgraded_df_lf.head()